In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline

*Notice that different part of code may need different configration, according to different data preprocessing method

#Read and analysis data

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp4432/dont-overfit-ii/test.csv")
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/comp4432/dont-overfit-ii/train.csv")
# Generate the column mapping dictionary
column_mapping = {col: f'{col}' for col in test.columns}

# Rename the columns using the mapping dictionary
test.rename(columns=column_mapping, inplace=True)
dataset.rename(columns=column_mapping, inplace=True)
# display(test)z

In [ ]:
import matplotlib.pyplot as plt

# Create a list of labels for the histogram
labels = ['Test Samples', 'Training Samples']

# Create a list of counts for each category
counts = [test.shape[0], dataset.shape[0]]

# Plotting the histogram
plt.bar(labels, counts)
plt.xlabel('Sample Type')
plt.ylabel('Number of Samples')
plt.title('Distribution of Training and Test Samples')
plt.show()

In [ ]:
import numpy as np
np.sum(dataset["target"] == 0)

In [ ]:
# import matplotlib.pyplot as plt
# # Example DataFrame
# data = pd.DataFrame({'Column1': [1, 4, 7, 3, 5, 2, 6, 8, 9, 3]})
# i=0
# j=0
# fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(15, 15))
# for col in dataset.columns[2:]:
#   if(i>5):
#     break
#   if(j>=4):
#     i+=1
#     j%=4
#   # Plot histogram of a column
#   axes[i, j].hist(dataset[col])
#   axes[i, j].set_title(col)
#   j+=1
#   # plt.ylabel('Frequency')
#   # plt.title(col)
#   # plt.show()

In [ ]:
dataset.describe().loc["std"].nlargest(2)

In [ ]:
dataset.describe().loc["std"].min()

In [ ]:
print(test.drop("id",axis=1).values.mean().sum()/300)
print(test.drop("id",axis=1).values.std().sum()/300)

In [ ]:
print(dataset.mean().sum()/300)
print(dataset.std().sum()/300)

In [ ]:
dataset = dataset.drop("id", axis=1)
test = test.drop("id", axis=1)

#*Training part(A compare the result of different simple models)

Scale the dataset

In [ ]:
from sklearn.preprocessing import RobustScaler

# dataset_x = RobustScaler().fit_transform(np.concatenate((dataset.drop("target",axis=1), test), axis=0))
# test= dataset_x[250:]
# dataset_x = dataset_x[:250]
# #add 0.1 as noise to make the model not overfitting
# dataset_x += np.random.normal(0, 0.1, dataset_x.shape)
dataset_x = RobustScaler().fit_transform(dataset.drop("target",axis=1))
test = RobustScaler().fit_transform(test)
dataset_x=pd.DataFrame(dataset_x)
dataset_x["target"]=dataset["target"]
dataset=dataset_x
test.shape
test=pd.DataFrame(test)

Prepare train and test data

In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE
from collections import Counter

train, validation = train_test_split(dataset, test_size=0.1, random_state=69)
# train, validation = train_test_split(dataset, test_size=0.2, random_state=42)
train_y=train["target"]
train_x=train.drop("target",axis=1)
val_y=validation["target"]
val_x=validation.drop("target",axis=1)
# # Identify minority class samples
# minority_samples = train[train["target"] == 1]

# # Determine the desired number of duplicates
# desired_duplicates = 100  # Choose the number of duplicates you want to create

# # Create duplicates by randomly sampling from the minority class samples
# duplicated_samples = minority_samples.sample(n=desired_duplicates, replace=True)

# # Concatenate the duplicated samples with the original training set
# train = pd.concat([train, duplicated_samples], axis=0, ignore_index=True).sample(frac=1, random_state=42)

normalization

In [ ]:
train_x

# Feature analysis

In [ ]:
# %pip install eli5
import eli5
from sklearn.ensemble import ExtraTreesClassifier
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Train a logistic regression model
# model = ExtraTreesClassifier()
model=LogisticRegression()
model.fit(train_x, train_y)

# # Use eli5 to get feature importance rankings
# # feature_names = ['Feature A', 'Feature B', 'Feature C', 'Feature D', 'Feature E']
feature_names = [str(feature) for feature in train_x.columns]
feature_importances = eli5.show_weights(model, feature_names=feature_names,top=50)

# # # Display the feature importance rankings
display(feature_importances)
# # Train a logistic regression model

# # Use permutation importance from eli5 to get feature importances
# perm = PermutationImportance(model).fit(train_x,train_y)
# feature_importances = eli5.show_weights(perm)
# display(eli5.show_weights(perm))

# # # Get a binary mask of selected features based on importance threshold
# # threshold = 0.1  # Adjust as needed
# mask = perm.feature_importances_ > threshold

# Select features based on the importance threshold
# X_train_selected = SelectFromModel(model, threshold=threshold).fit_transform(train_x, train_y)
# X_val_selected = SelectFromModel(model, threshold=threshold).transform(val_x)
# X_test_selected = SelectFromModel(model, threshold=threshold).transform(test)
# train_x=pd.DataFrame(X_train_selected)
# val_x=pd.DataFrame(X_val_selected)
# test=pd.DataFrame(X_test_selected)

In [ ]:
train_x.shape

Resample

In [ ]:
from imblearn.over_sampling import SMOTE
# Assuming 'X' is your feature matrix and 'y' is the corresponding label array
smote = SMOTE(sampling_strategy='minority', n_jobs=-1)
train_x, train_y = smote.fit_resample(train_x,train_y)
print('Resampled dataset shape %s' % Counter(train_y))

In [ ]:
train_x

In [ ]:
train_x.shape

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Create an SVM model
model = SVC()

# Train the model on the training data
model.fit(train_x, train_y)

# Make predictions on the test data
y_pred = model.predict(val_x)

# Evaluate the accuracy of the model
accuracy = accuracy_score(val_y, y_pred)
print("Accuracy:", accuracy)

In [ ]:
model.predict(val_x)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

X = dataset.drop("target",axis=1)
y = dataset["target"]
# model = ExtraTreesClassifier()
model = LogisticRegression()
# model = SVC()
# model = KNeighborsClassifier()
# model = RandomForestClassifier()
# Define the number of folds for cross-validation
k = 10

# Perform K-fold cross-validation
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Perform cross-validation and compute the mean accuracy
accuracy_scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
mean_accuracy = accuracy_scores.mean()
# Perform K-fold cross-validation
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Perform cross-validation and compute the mean accuracy
accuracy_scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
mean_accuracy = accuracy_scores.mean()

# Print the accuracy for each fold and the mean accuracy
i=1
for score in accuracy_scores:
  print("Accuracy for fold {} : {}".format(i,score))
  i+=1
print(mean_accuracy)

In [ ]:
pred=model.predict(train_x)
accuracy = accuracy_score(train_y, pred)
print("Accuracy:", accuracy)
display(pred)
display(train_y.shape)

#Feature selection

In [ ]:
# from sklearn.feature_selection import RFE
# from sklearn.datasets import make_regression
# from sklearn.ensemble import ExtraTreesClassifier
# # Create an instance of the model you want to use (e.g., Linear Regression)
# model = LogisticRegression(solver="liblinear", penalty="l1", C=0.1)

# # Create an RFE object and specify the model and number of features to select
# rfe = RFE(estimator=model, n_features_to_select=50)

# # Fit the RFE object to the data
# rfe.fit(train_x, train_y)

# # Get the selected features
# selected_features = rfe.support_
# selected_indices = [i for i, selected in enumerate(selected_features) if selected]

# print("Selected Features:")
# for i in selected_indices:
#     print(f"Feature {i+1}")
# train_x = train_x.iloc[:, selected_indices]
# val_x = val_x.iloc[:, selected_indices]
# selected_test = test.drop("id",axis=1).iloc[:, selected_indices]

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# Apply L1 regularization using Lasso
lasso = Lasso(alpha=0.001)  # adjust the regularization strength with the alpha parameter
lasso.fit(train_x, train_y)

# Access the coefficients
coefficients = lasso.coef_
coefficients
# Obtain the regularized dataset
selected_features = np.where(lasso.coef_ != 0)[0]
train_x = train_x.iloc[:, selected_features]
val_x = val_x.iloc[:, selected_features]
test_x =test.iloc[:, selected_features]

In [ ]:
import pandas as pd

# Assuming your dataset is stored in a pandas DataFrame called 'data'
correlation_matrix = train_x.corr()
threshold = 0.5  # Set the correlation threshold

# Find highly correlated feature pairs
highly_correlated = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            highly_correlated.add(colname)
# Drop the highly correlated features from the original dataset
train_x = train_x.drop(highly_correlated, axis=1)
test_x= test.drop(highly_correlated, axis=1)
val_x = val_x.drop(highly_correlated, axis=1)

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# Assuming your dataset is stored in a Pandas DataFrame called 'df'
# and the target variable is named 'target'

# Separate the features and target variable
X = train_x
y = train_y

# Calculate the information gain (mutual information) for each feature
information_gains = mutual_info_classif(X, y)

# Create a DataFrame to store the feature names and their corresponding information gains
feature_scores = pd.DataFrame({'Feature': X.columns, 'Information Gain': information_gains})

# Sort the features based on their information gains in descending order
sorted_features = feature_scores.sort_values('Information Gain', ascending=False)

# Print the top 10 features with the highest information gain
# top_features = sorted_features.head(150)
# print(top_features)
# Set a threshold to select the top features with high information gain
threshold = 0.01  # Adjust the threshold as needed

# Filter the features based on the threshold
selected_features = sorted_features[sorted_features['Information Gain'] > threshold]

# Obtain the final list of selected feature names
selected_feature_names = selected_features['Feature'].tolist()

# Print the selected features
print(len(selected_feature_names))

In [ ]:
val_x=val_x[selected_feature_names]
train_x=train_x[selected_feature_names]
selected_test=test.iloc[:,selected_feature_names]

In [ ]:
train_x.shape

# Outlier elimination

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the z-scores for each feature
z_scores = (train_x - np.mean(train_x, axis=0)) / np.std(train_x, axis=0)

# Set the threshold for outliers
threshold = 3.7

# Find the outliers for each attribute
outliers = np.where(np.abs(z_scores) > threshold)

# Plot the z-scores for each attribute
plt.figure(figsize=(15, 20))
plt.boxplot(z_scores)
plt.xlabel('Attribute Index')
plt.ylabel('Z-Score')
plt.title('Z-Scores for Each Attribute')
plt.show()
plt.show()
# Flatten the outliers indices
outliers = np.any(np.abs(z_scores) > threshold, axis=1)
# Remove the outliers from the dataset and labels
train_x = train_x[~outliers]
train_y = train_y[~outliers]

In [ ]:
highly_correlated

In [ ]:
model.fit(train_x, train_y)
y_pred = model.predict(val_x)
print(y_pred)

# Voting classifier(Train with network)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

def build_simple_nn(input_dim):
    # Define the architecture of the neural network
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_shape=(input_dim,)))  # Input layer
    model.add(Dense(units=1, activation='sigmoid'))  # Output layer (one unit for binary classification)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
input_dim = 300

# Build the simple neural network
model = build_simple_nn(input_dim)

# Train the model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(val_x, val_y)

Try to use a NN to get voting result

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.utils import to_categorical
import numpy as np
# Convert the binary numbers to a 2D NumPy array
train_encode = np.array(train_y)

hot_train_y = [to_categorical(label, num_classes=2).tolist() for label in train_y]
lr=0.1
# Load your dataset (X and y) or use your own dataset
X, y = train_x, train_y
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = SVC(probability=True)
clf4 = ExtraTreesClassifier()
clf5 = KNeighborsClassifier()
#build the voting classififer
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3),('Extree',clf4),('KNN',clf5)], voting='soft', weights=[1, 1, 0.5, 1, 0.3])
# Define the number of folds for K-fold cross-validation
k = 10

# Initialize a list to store the accuracies for each fold
accuracies = []

# Initialize the K-fold cross-validator
kf = KFold(n_splits=k)
# Create the model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(5,))) # Single layer with 2 output nodes
model.add(Dense(units=128, activation='relu'))  # Output layer (one unit for binary classification)
model.add(Dense(units=2, activation="softmax"))  # Output layer (one unit for binary classification)
# model.add(Dense(units=1, activation='sigmoid'))  # Output layer (one unit for binary classification)
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
# model.fit(X, y, epochs=10, batch_size=32)

# Make predictions
# predictions = model.predict(X)
# Iterate over the folds
y_all=np.array([])
stacked= np.empty((0, 5, 1))
for train_index, val_index in kf.split(range(train.shape[0])):
    # Split the data into training and validation sets for the current fold
    result=[]
    # print(train_index.shape)
    # print(val_index.shape)
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Train the model on the training set
    voting_clf.fit(X_train, y_train)

    # Evaluate the model on the validation set
    y_pred = voting_clf.predict_proba(X_val)
    for estimator in voting_clf.estimators_:
      pred=estimator.predict_proba(X_val)[:,1]
      result.append(pred)
    # print(result)
    result=np.transpose(np.array(result)).reshape(-1, 5,1)
    stacked = np.concatenate((stacked, result), axis=0)
    # print(stacked)
    y_all = np.concatenate((y_all, y_val))
# result=np.transpose(np.array(result)).reshape(-1, 5,1)
# modified_result = np.hstack((result, np.ones((result.shape[0], 1))))[:, :,np.newaxis]
hot_train_y = [to_categorical(label, num_classes=2).tolist() for label in y_all]
model.fit(stacked, np.array(hot_train_y))

# Calculate the average accuracy across all folds
# average_accuracy = sum(accuracies) / k

# Print the average accuracy
# print("Average Accuracy:", average_accuracy)

In [ ]:
from sklearn.metrics import roc_curve, auc
result=[]
for estimator in voting_clf.estimators_:
  pred=estimator.predict_proba(train_x)[:,1]
  result.append(pred)
result=np.transpose(np.array(result)).reshape(-1, 5,1)
pred=model.predict(result)
# hot_train_y = [to_categorical(label, num_classes=2).tolist() for label in train_y]
fpr, tpr, thresholds = roc_curve(np.argmax(pred, axis=1), train_y)

# Compute the Area Under the ROC Curve (AUC-ROC)
auc_score = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], 'k--')  # Plot the diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
pred

In [ ]:
model.fit(stacked, np.array(hot_train_y))

In [ ]:
stacked.shape

In [ ]:
result

In [ ]:
np.count_nonzero(dataset["target"]==0)

try to use panalty

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import statistics
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import SGDClassifier
lr=0.1
# Load your dataset (X and y) or use your own dataset
X, y = train_x, train_y
clf1 = LogisticRegression(max_iter=100, class_weight='balanced', penalty='l2', C=0.1, solver='liblinear')
# clf2 = RandomForestClassifier()
clf2= LogisticRegression(class_weight='balanced', solver='liblinear', penalty ='l1', C= 0.1, max_iter=10000)
clf3 = LogisticRegression(penalty="l2",C=0.8)
clf4 = GradientBoostingClassifier()
# clf4 = GaussianNB()
meta_classifier=GaussianNB()
# clf5 = KNeighborsClassifier()
clf5= SGDClassifier(eta0=1, max_iter=100, tol=0.0001, alpha=0.01, l1_ratio=0.0, learning_rate='adaptive', loss='log', penalty='elasticnet')
#build the voting classififer
# voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3),('Extree',clf4),('KNN',clf5)], voting='soft', weights=[1, 1, 1, 1, 1])
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf5)], voting='soft', weights=[1, 1, 1])
# Define the number of folds for K-fold cross-validation
k = 20
# voting_clf = RFE(estimator=clf2, n_features_to_select=150,importance_getter='feature_importances_')
# Initialize a list to store the accuracies for each fold
accuracies = []

# Initialize the K-fold cross-validator
kf = KFold(n_splits=k,shuffle=True)
result=[]
y_all=np.array([])
weight=[1.0,1.0,1.0]
j=0
print("The classsifiers are LogisticRegression, RFC, SVC, ExtraTreesClassifier, KNeighborsClassifier")
for train_index, val_index in kf.split(range(train.shape[0])):
    j+=1
    # Split the data into training and validation sets for the current fold
    # print(train_index.shape)
    # print(val_index.shape)
    result=[]
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Train the model on the training set
    voting_clf.fit(X_train, y_train)

    # Evaluate the model on the validation set
    accuracies=[]
    # y_pred = voting_clf.predict_proba(X_val)
    for estimator in voting_clf.estimators_:
      pred=estimator.predict_proba(X_val)[:,1]
      result.append(pred)
      # print(pred)
      # print(y_val)
      auc_roc = roc_auc_score(y_val, pred)
      accuracies.append(auc_roc)
    threshold_auc=statistics.median(accuracies)
    penalty_factor = max(0, threshold_auc - auc_roc)
    i=0
    for auc_roc,w in zip(accuracies,weight):
      penalty_factor = max(0, threshold_auc - auc_roc)
      # penalized_auc_roc = auc_roc * penalty_factor
      weight[i]=w-w * penalty_factor
      i+=1
    total = sum(weight)
    # print(y_val)
    normalized_weights = [w / total for w in weight]

    rounded_numbers = []
    for number in normalized_weights:
        rounded_number = round(number, 2)
        formatted_number = "{:.2f}".format(rounded_number)
        rounded_numbers.append(formatted_number)
    print("Fold {},the updated weight(normalized) are {}".format(j,rounded_numbers))
#   # Calculate the sum of the weights

total = sum(weight)
# Normalize the weights
normalized_weights = [w / total for w in weight]
voting_clf.weights=normalized_weights
#       result.append(pred)
#     y_all = np.concatenate((y_all, y_val))
# result=np.transpose(np.array(result)).reshape(-1, 5,1)
# modified_result = np.hstack((result, np.ones((result.shape[0], 1))))[:, :,np.newaxis]
# model.fit(result, y_all)

# Calculate the average accuracy across all folds
# average_accuracy = sum(accuracies) / k

# Print the average accuracy
# print("Average Accuracy:", average_accuracy)

Using the Meta-classifier


In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
import statistics
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import SGDClassifier
lr=0.1
# Load your dataset (X and y) or use your own dataset
X, y = train_x, train_y
clf1 = LogisticRegression(max_iter=100, class_weight='balanced', penalty='l2', C=0.1, solver='liblinear')
# clf2 = RandomForestClassifier()
clf2= LogisticRegression(class_weight='balanced', solver='liblinear', penalty ='l1', C= 0.1, max_iter=10000)
clf3 = LogisticRegression(penalty="l2",C=0.8)
clf4 = ExtraTreesClassifier()
# clf4 = GaussianNB()
meta_classifier=GaussianNB()
# clf5 = KNeighborsClassifier()
clf5= SGDClassifier(eta0=1, max_iter=100, tol=0.0001, alpha=0.01, l1_ratio=0.0, learning_rate='adaptive', loss='log', penalty='elasticnet')
#build the voting classififer
# voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3),('Extree',clf4),('KNN',clf5)], voting='soft', weights=[1, 1, 1, 1, 1])
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf5)], voting='soft', weights=[1, 1, 1])
voting_clf.weights=normalized_weights
# Define the number of folds for K-fold cross-validation
k = 75
# voting_clf = RFE(estimator=clf2, n_features_to_select=150,importance_getter='feature_importances_')
# Initialize a list to store the accuracies for each fold
accuracies = []

# Initialize the K-fold cross-validator
kf = KFold(n_splits=k)
result=[]
y_all=np.array([])
weight=[1.0,1.0,1.0,1.0,1.0]
j=0
print("The classsifiers are LogisticRegression, RFC, SVC, ExtraTreesClassifier, KNeighborsClassifier")
for train_index, val_index in kf.split(range(train.shape[0])):
    j+=1
    # Split the data into training and validation sets for the current fold
    # print(train_index.shape)
    # print(val_index.shape)
    result=[]
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Train the model on the training set
    # if(j==1):
    voting_clf.fit(X_train, y_train)
    # else:
      # for estimator in voting_clf.estimators_:
        # estimator.partial_fit(X_train,y_train,classes=np.unique(y))
    # Evaluate the model on the validation set
    accuracies=[]
    # y_pred = voting_clf.predict_proba(X_val)
    i=0
    for estimator in voting_clf.estimators_:
      pred=estimator.predict_proba(X_val)[:,1]*voting_clf.weights[i]
      i+=1
      result.append(pred)
    result1=np.transpose(np.array(result)).reshape(-1, 3)
    if(j==1):
      meta_classifier.fit(result1,y_val)
    else:
      meta_classifier.partial_fit(result1, y_val)
'''
Using AUC-ROC to do the weight updating
'''
#       auc_roc = roc_auc_score(val_y, pred)
#       accuracies.append(auc_roc)
#     threshold_auc=statistics.median(accuracies)
#     penalty_factor = max(0, threshold_auc - auc_roc)
#     i=0
#     for auc_roc,w in zip(accuracies,weight):
#       penalty_factor = max(0, threshold_auc - auc_roc)
#       # penalized_auc_roc = auc_roc * penalty_factor
#       weight[i]=w-w * penalty_factor
#       i+=1
#     total = sum(weight)
#     # print(y_val)
#     normalized_weights = [w / total for w in weight]

#     rounded_numbers = []
#     for number in normalized_weights:
#         rounded_number = round(number, 2)
#         formatted_number = "{:.2f}".format(rounded_number)
#         rounded_numbers.append(formatted_number)
#     print("Fold {},the updated weight(normalized) are {}".format(j,rounded_numbers))
#   # Calculate the sum of the weights

# total = sum(weight)
# # Normalize the weights
# normalized_weights = [w / total for w in weight]
# voting_clf.weights=normalized_weights
#       result.append(pred)
#     y_all = np.concatenate((y_all, y_val))
# result=np.transpose(np.array(result)).reshape(-1, 5,1)
# modified_result = np.hstack((result, np.ones((result.shape[0], 1))))[:, :,np.newaxis]
# model.fit(result, y_all)

# Calculate the average accuracy across all folds
# average_accuracy = sum(accuracies) / k

# Print the average accuracy
# print("Average Accuracy:", average_accuracy)

In [ ]:
result=[]
i=0
for estimator in voting_clf.estimators_:
  pred=estimator.predict_proba(val_x)[:,1]*voting_clf.weights[i]
  i+=1
  result.append(pred)
result=np.transpose(np.array(result)).reshape(-1, 3)
pred=meta_classifier.predict(result)
accuracy = accuracy_score(val_y, pred)
display(accuracy)
display(val_y)
display(pred)

In [ ]:
test.shape

In [ ]:
result=[]
for estimator in voting_clf.estimators_:
  pred=estimator.predict(val_x)
  result.append(pred)
  accuracy = accuracy_score(val_y, pred)
  print(accuracy)
#   print(pred)
# print(val_y)
i=0
vote=[0 for _ in range(len(train_y))]
for r in result:
  # print(r)
  vote+=r*voting_clf.weights[i]
pred=voting_clf.predict(val_x)
accuracy = accuracy_score(val_y, pred)
# print(accuracy)
# result=np.transpose(np.array(result)).reshape(-1, 5,1)
# # modified_result = np.hstack((result, np.ones((result.shape[0], 1))))[:, :,np.newaxis]
# pred=voting_clf.predict(val_x)
# pred
# print(list(val_y,type=uint8))
# display(accuracy)
# display(pred)
# display(list(val_y))
print(accuracy)
# print(y_pred)
# print(val_y)

In [ ]:
result

In [ ]:
val_y

In [ ]:
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(voting_clf.predict(val_x), val_y)

# Compute the Area Under the ROC Curve (AUC-ROC)
auc_score = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], 'k--')  # Plot the diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()
print(y_pred)
print(val_y)

In [ ]:
display(voting_clf.predict(val_x))
display(val_y)

Visualization of prediction result

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming you have computed the true labels and predicted probabilities
# true_labels: true labels of the validation or test data
# predicted_probs: predicted probabilities for the positive class

for r in result:
  # Compute the False Positive Rate (FPR), True Positive Rate (TPR), and thresholds
  fpr, tpr, thresholds = roc_curve(val_y, r)

  # Compute the Area Under the ROC Curve (AUC-ROC)
  auc_score = auc(fpr, tpr)

  # Plot the ROC curve
  plt.figure()
  plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % auc_score)
  plt.plot([0, 1], [0, 1], 'k--')  # Plot the diagonal line
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend(loc='lower right')
  plt.show()

In [ ]:
r

In [ ]:
result=[]
for estimator in voting_clf.estimators_:
  pred=estimator.predict_proba(test.drop("id",axis=1))[:,1]
  result.append(pred)
result=np.transpose(np.array(result)).reshape(-1, 5,1)
# modified_result = np.hstack((result, np.ones((result.shape[0], 1))))[:, :,np.newaxis]
pred=model.predict(result)

In [ ]:
np.sum(pred > 0.5)

In [ ]:
# Evaluate the model on the validation set
y_pred = voting_clf.predict_proba(val_x)
result=[]
for estimator in voting_clf.estimators_:
  pred=estimator.predict_proba(val_x)[:,1]
  result.append(pred)
result=np.transpose(np.array(result)).reshape(-1, 5, 1)
model.predict(result)

In [ ]:
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Assuming you have your feature matrix X and target variable y
# Create individual classifiers
clf1 = LogisticRegression(random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = SVC(random_state=42)
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('svc', clf3)], voting='hard', weights=[1, 1, 1])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM model
model = svm.SVC(probability=True)  # Set probability=True to enable probability estimates

# Train the model on the training data
model.fit(X_train, y_train)

# Predict the probabilities for the positive class
y_prob = model.predict_proba(X_test)[:, 1]

# Calculate the AUC-ROC
auc_roc = roc_auc_score(y_test, y_prob)

print("AUC-ROC:", auc_roc)

# Obtain test prediction result

In [ ]:
# Perform majority voting to get the final prediction
# voting_predictions = voting_clf.predict(selected_test).flatten()

# Prepare the submission file
# submission = pd.DataFrame({'id': test['id'], 'target': voting_predictions})

# Save the submission file as a CSV
# submission.to_csv('/content/drive/MyDrive/Colab Notebooks/comp4432/dont-overfit-ii/submission.csv', index=False)
# x_test=scaler.fit_transform(test.drop("id",axis=1))
result=[]
i=0
for estimator in voting_clf.estimators_:
  pred=estimator.predict_proba(test_x)[:,1]*voting_clf.weights[i]
  i+=1
  result.append(pred)
result=np.transpose(np.array(result)).reshape(-1, 3)
pred=meta_classifier.predict_proba(result)[:,1]
# pred = pd.DataFrame(pred)
pred = pd.DataFrame(voting_clf.predict_proba(test_x)[:,1])
pred.index += 250
pred.columns = ['target']
pred.to_csv('/content/drive/MyDrive/Colab Notebooks/comp4432/dont-overfit-ii/submission.csv', index_label='id', index=True)

In [ ]:
voting_clf.weights

In [ ]:
np.count_nonzero(voting_clf.predict(test_x) == 0)

In [ ]:
model.predict(test)